In [ ]:
# Mount Google Drive so that training data can be used
def mount_drive():
  from google.colab import drive
  drive.mount('/content/drive')

mount_drive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# load data
import pandas as pd

song_data = pd.read_csv("/content/drive/Shareddrives/Cop Detectors /Class Work/song_data/tracks_features.csv")

In [ ]:
print(song_data.columns)

# process data to fit into feature set
feature_list = ['acousticness',
                'danceability',
                'energy',
                'key',
                'liveness',
                'loudness',
                'tempo']
features = song_data[feature_list]
print(features.describe())

# normalize data
features = tf.keras.utils.normalize(features)


# reshape data into a training set
x_train = features.sample(frac=0.8, random_state=42)
x_test = features.drop(x_train.index)

x_train = x_train.values
x_test = x_test.values
print(x_train.shape)
print(x_test.shape)

Index(['id', 'name', 'album', 'album_id', 'artists', 'artist_ids',
       'track_number', 'disc_number', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms',
       'time_signature', 'year', 'release_date'],
      dtype='object')
       acousticness  danceability        energy           key      liveness  \
count  1.204025e+06  1.204025e+06  1.204025e+06  1.204025e+06  1.204025e+06   
mean   4.467511e-01  4.930565e-01  5.095363e-01  5.194151e+00  2.015994e-01   
std    3.852014e-01  1.896694e-01  2.946839e-01  3.536731e+00  1.804591e-01   
min    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
25%    3.760000e-02  3.560000e-01  2.520000e-01  2.000000e+00  9.680000e-02   
50%    3.890000e-01  5.010000e-01  5.240000e-01  5.000000e+00  1.250000e-01   
75%    8.610000e-01  6.330000e-01  7.660000e-01  8.000000e+00  2.450000e-01   
max    9.96000

In [ ]:
# Unsupervised Model
# In this case, an autoencoder.

# We will take in the following features (for now):
# acousticness (0.0 to 1.0)
# danceability (0.0 to 1.0)
# energy (0.0 to 1.0)
# key (-1 to 11), 0 = C
# liveness (0.0 to 1.0?)
# loudness (-60dB to 0 dB)
# tempo (BPM)



# For now, using this dataset from Kaggle:
# It has 1.2 million Spotify songs' features
# https://www.kaggle.com/datasets/rodolfofigueroa/spotify-12m-songs?resource=download


# Each feature will be a node in the first layer
# Each new layer will reduce the number of nodes by 1
# So there will be a 7 node layer, then a 6 node layer, then 5, and so on
# I'm not currently sure whether this approach is best for encoding, because
# it could be the case that much information is lost if we go down to 1 layer,
# to the point of it becoming useless.
# Also, stepping down faster (like halving the number of nodes) might be better.
# Then there will be a sequence of decoding the information back up to 7 dimensions
# This is so that we can verify that the encoding maintained the original information
# Otherwise we could not train the model weights.

# We can play with and research different architectures.

import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

# using this resource: https://towardsdatascience.com/unsupervised-machine-learning-example-in-keras-8c8bf9e63ee0
# and https://www.tensorflow.org/api_docs/python/tf/keras/Sequential
# and MAINLY https://www.tensorflow.org/tutorials/generative/autoencoder

# latent dimensions: The number of dimensions in the compressed representation

class Autoencoder(Model):
  def __init__(self):
    super(Autoencoder, self).__init__()
    self.encoder = tf.keras.Sequential([
        layers.Flatten(),
        layers.Dense(7, activation='relu'),
        layers.Dense(6, activation='relu'),
        layers.Dense(5, activation='relu'),
        layers.Dense(4, activation='relu'),
        layers.Dense(3, activation='relu'),
        layers.Dense(2, activation='relu'),
        layers.Dense(1, activation='relu'),
        
    ])
    self.decoder = tf.keras.Sequential([
        layers.Dense(1, activation='sigmoid'),
        layers.Dense(2, activation='sigmoid'),
        layers.Dense(3, activation='sigmoid'),
        layers.Dense(4, activation='sigmoid'),
        layers.Dense(5, activation='sigmoid'),
        layers.Dense(6, activation='sigmoid'),
        layers.Dense(7, activation='sigmoid')
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder()

In [ ]:
autoencoder.compile(optimizer='adam', 
                    loss=losses.MeanSquaredError(),
                    metrics=[["accuracy",]])

In [ ]:
autoencoder.fit(x_train, x_train,
                epochs=10,
                shuffle=True,
                validation_split=0.2)

Epoch 1/10
24081/24081 [==============================] - 142s 6ms/step - loss: 0.0057 - accuracy: 0.9977 - val_loss: 0.0034 - val_accuracy: 0.9978
Epoch 2/10
 5753/24081 [======>.......................] - ETA: 1:36 - loss: 0.0035 - accuracy: 0.9977

KeyboardInterrupt: ignored

In [ ]:
# TODO: PCA (preferred) or t-SNE on the features + the outputted feature(s) 
# from the unsupervised model, for dimensionality reduction

In [ ]:
# TODO: Nearest Neighbor Search algorithm in feature space